In [1]:
import pmagpy.pmag as pmag
import pmagpy.pmagplotlib as pmagplotlib
import pmagpy.ipmag as ipmag
import pmagpy.contribution_builder as cb
from pmagpy import convert_2_magic as convert
import matplotlib.pyplot as plt # our plotting buddy
import numpy as np # the fabulous NumPy package
import pandas as pd # and of course Pandas
# test if Basemap and/or cartopy is installed
has_basemap, Basemap = pmag.import_basemap()
has_cartopy, Cartopy = pmag.import_cartopy()
# This allows you to make matplotlib plots inside the notebook.  
%matplotlib inline 
from IPython.display import Image
import os

from importlib import reload

from pmagpy.mapping import map_magic    




In [113]:
def sites_extract(site_file='sites.txt',directions_file='directions.xls',
                  intensity_file='intensity.xls',info_file='site_info.xls',
                  output_dir_path='./',input_dir_path='./',latex=False):
    """
    Extracts directional and/or intensity data from a MagIC 3.0 format sites.txt file.
    Default output format is an Excel file. 
    Optional latex format longtable file which can be uploaded to Overleaf or 
    typeset with latex on your own computer.  
    
    Parameters
    ___________
    site_file : str
         input file name
    directions_file : str
          output file name for directional data
    intensity_file : str
          output file name for intensity data
    site_info : str
          output file name for site information (lat, lon, location, age....)
    output_dir_path : str
          path for output files
    input_dir_path : str
          path for intput file if different from output_dir_path (default is same)
    latex : boolean
           if True, output file should be latex formatted table with a .tex ending
           
    Return :
        [True,False], error type : True if successful
        
    Effects : 
        writes cvs or latex formatted tables for use in publications
    """
    if not input_dir_path:
        input_dir_path = output_dir_path
    try:
        fname = pmag.resolve_file_name(site_file, input_dir_path)
    except IOError:
        print("bad site file name")
        return False, "bad site file name"
    sites_df=pd.read_csv(fname,sep='\t',header=1)
# do directional stuff first
    # a few things need cleaning up
    dir_df=sites_df.dropna(subset=['dir_dec','dir_inc']) # delete blank directions
    # sort by absolute value of vgp_lat in order to eliminate duplicate rows for
    # directions put in by accident on intensity rows
    if len(dir_df)>0:

        DirCols = ["Site", "TC (%)", "Dec.", "Inc.", "N", "k    ", "R", "a95"]

        dir_file=pmag.resolve_file_name(directions_file,output_dir_path)
        dir_df['dir_n_samples']=dir_df['dir_n_samples'].values.astype('int')
        dir_df['dir_tilt_correction']=dir_df['dir_tilt_correction'].values.astype('int')
        has_vgps=False
        if 'vgp_lat' in dir_df.columns:
            test_vgp=dir_df.dropna(subset=['vgp_lat','vgp_lon'])
            if len(test_vgp)>0: has_vgps=True
        if has_vgps:
            dir_df['vgp_lat_abs']=dir_df.vgp_lat.abs() 
            dir_df.sort_values(by=['site','vgp_lat_abs'],ascending=False,inplace=True)
            dir_df=dir_df[['site','dir_tilt_correction','dir_dec','dir_inc',\
                'dir_n_samples','dir_k','dir_r','dir_alpha95','vgp_lat','vgp_lon']]
    # this will take the first record for each site's directions (including VGP lat if present)
            DirCols.append("VGP Lat")
            DirCols.append("VGP Long")
            dir_df.drop_duplicates(subset=['dir_dec','dir_inc','site'],inplace=True)
        else:
            dir_df.drop_duplicates(subset=['dir_dec','dir_inc','site'],inplace=True)
            dir_df=dir_df[['site','dir_tilt_correction','dir_dec','dir_inc',\
                   'dir_n_samples','dir_k','dir_r','dir_alpha95']]   
        dir_df.columns=DirCols 
        dir_df.sort_values(by=['Site'],inplace=True,ascending=True)
        if latex:
            directions_out = open(dir_file, 'w+', errors="backslashreplace")
            directions_out.write('\documentclass{article}\n')
            directions_out.write('\\usepackage{booktabs}\n')
            directions_out.write('\\usepackage{longtable}\n')
            directions_out.write('\\begin{document}')
            directions_out.write(dir_df.to_latex(index=False,longtable=True,multicolumn=False))
            directions_out.write('\end{document}\n')
            directions_out.close()
        else:
            dir_df.to_excel(dir_file,index=False)
    else: 
        print ("No directional data for ouput.")
# now for the intensities
    has_vadms,has_vdms=False,False
    int_df=sites_df.dropna(subset=['int_abs'])
    int_df['int_n_samples']=int_df['int_n_samples'].values.astype('int')
    if len(int_df)>0:
        int_df['int_abs_uT']=1e6*int_df.int_abs.values # convert to uT
        int_df['int_abs_sigma_uT']=1e6*int_df.int_abs_sigma.values # convert to uT
        int_df['int_abs_uT']=int_df['int_abs_uT'].values.astype('int')
        int_df['int_abs_sigma_uT']=int_df['int_abs_sigma_uT'].values.astype('int')
        int_df['int_abs_sigma_perc']=int_df['int_abs_sigma_perc'].values.astype('int')
        int_file=pmag.resolve_file_name(intensity_file,output_dir_path)

        IntCols = ["Site", "N", "B", "B sigma","sigma (%)"]
        if 'vadm' in int_df.columns:
            test_vadm=int_df.dropna(subset=['vadm'])
            if len(test_vadm)>0: 
                has_vadms=True

        if 'vdm' in int_df.columns:
            test_vdm=int_df.dropna(subset=['vdm'])
            if len(test_vdm)>0: has_vdms=True
       
        if has_vadms:
            IntCols.append("VADM")
            IntCols.append("VADM sigma")
        if has_vdms:
            IntCols.append("VDM")
            IntCols.append("VDM sigma")
        if not has_vadms and not has_vdms:
            int_df=int_df[['site','int_n_samples','int_abs_uT','int_abs_sigma_uT',\
                'int_abs_sigma_perc']]
        if has_vadms and not has_vdms:
            int_df.sort_values(by=['site','vadm'],ascending=False,inplace=True)
            int_df.drop_duplicates(subset=['int_abs_uT','site'],inplace=True)

            int_df['vadm_ZAm2']=1e-21*int_df.vadm.values
            int_df['vadm_sigma_ZAm2']=1e-21*int_df.vadm_sigma.values
            int_df=int_df[['site','int_n_samples','int_abs_uT','int_abs_sigma_uT',\
                'int_abs_sigma_perc','vadm_ZAm2','vadm_ZAm2_sigma']]
        if not has_vadms and  has_vdms:
            int_df.sort_values(by=['site','vdm'],ascending=False,inplace=True)
            int_df.drop_duplicates(subset=['int_abs_uT','site'],inplace=True)
            int_df['vdm_ZAm2']=1e-21*int_df.vdm.values()
            int_df['vdm_sigma_ZAm2']=1e-21*int_df.vdm_sigma.values()

            int_df=int_df[['site','int_n_samples','int_abs_uT','int_abs_sigma_uT',\
                'int_abs_sigma_perc','vdm_ZAm2','vdm_ZAm2_sigma']]
        if has_vadms and  has_vdms:
            int_df.sort_values(by=['site','vadm'],ascending=False,inplace=True)
            int_df.drop_duplicates(subset=['int_abs_uT','site'],inplace=True)
            int_df['vadm_ZAm2']=1e-21*int_df.vadm.values
            int_df['vadm_sigma_ZAm2']=1e-21*int_df.vadm_sigma.values
            int_df['vdm_ZAm2']=1e-21*int_df.vdm.values
            int_df['vdm_sigma_ZAm2']=1e-21*int_df.vdm_sigma.values
            int_df=int_df[['site','int_n_samples','int_abs_uT','int_abs_sigma_uT',\
                'int_abs_sigma_perc','vadm_ZAm2','vadm_sigma_ZAm2','vdm_ZAm2','vdm_sigma_ZAm2']]
        int_df.columns=IntCols 
        int_df.sort_values(by=['Site'],inplace=True,ascending=True)
        int_df.fillna(value='',inplace=True)
        if latex:
            intensities_out = open(int_file, 'w+', errors="backslashreplace")
            intensities_out.write('\documentclass{article}\n')
            intensities_out.write('\\usepackage{booktabs}\n')
            intensities_out.write('\\usepackage{longtable}\n')
            intensities_out.write('\\begin{document}')
            intensities_out.write(int_df.to_latex(index=False,longtable=True,multicolumn=False))
            intensities_out.write('\end{document}\n')
            intensities_out.close()
        else:
            int_df.to_excel(int_file,index=False)
    else: 
        print ("No intensity data for ouput.")
# site info
    nfo_df=sites_df.dropna(subset=['lat','lon']) # delete blank locations
    if len(nfo_df)>0:
        SiteCols = ["Site", "Location","Lat. (N)", "Long. (E)"]
        info_file=pmag.resolve_file_name(info_file,output_dir_path)


        test_age=nfo_df.dropna(subset=['age','age_sigma','age_unit'])
        if len(test_age)>0:                       
            SiteCols.append("Age ")
            SiteCols.append("Age sigma")
            SiteCols.append("Units")
            nfo_df=nfo_df[['site','location','lat','lon','age','age_sigma','age_unit']]
        else:
            nfo_df=nfo_df[['site','location','lat','lon']]
        nfo_df.drop_duplicates(inplace=True)
        nfo_df.columns=SiteCols
        nfo_df.fillna(value='',inplace=True) 
        if latex:
            info_out = open(info_file, 'w+', errors="backslashreplace")
            info_out.write('\documentclass{article}\n')
            info_out.write('\\usepackage{booktabs}\n')
            info_out.write('\\usepackage{longtable}\n')
            info_out.write('\\begin{document}')
            info_out.write(nfo_df.to_latex(index=False,longtable=True,multicolumn=False))
            info_out.write('\end{document}\n')
            info_out.close()
        else:
            nfo_df.to_excel(info_file,index=False)
    else: 
        print ("No location information  for ouput.")
       

In [114]:
#latex way:
sites_extract(directions_file='directions.tex',intensity_file='intensities.tex',
              output_dir_path='/users/ltauxe/Desktop',info_file='site_info.tex',
              input_dir_path='data_files/3_0/McMurdo',latex=True)

In [164]:
#xls way:
sites_extract(output_dir_path='/users/ltauxe/Desktop',
              input_dir_path='data_files/3_0/McMurdo')

In [78]:
def criteria_extract(crit_file='criteria.txt',output_file='criteria.xls',
                  output_dir_path='./',input_dir_path='./',latex=False):
    """
    Extracts criteria  from a MagIC 3.0 format criteria.txt file.
    Default output format is an Excel file. 
    typeset with latex on your own computer.  
    
    Parameters
    ___________
    crit_file : str
         input file name
    input_dir_path : str
          path for intput file if different from output_dir_path (default is same)
    latex : boolean
           if True, output file should be latex formatted table with a .tex ending
           
    Return :
        [True,False],  data table error type : True if successful
        
    Effects : 
        writes cvs or latex formatted tables for use in publications
    """
    if not input_dir_path:
        input_dir_path = output_dir_path
    try:
        fname = pmag.resolve_file_name(crit_file, input_dir_path)
    except IOError:
        print("bad criteria file name")
        return False, "bad site file name"
    crit_df=pd.read_csv(fname,sep='\t',header=1)
    if len(crit_df)>0:
        out_file=pmag.resolve_file_name(output_file,output_dir_path)


        s=crit_df['table_column'].str.split(pat='.',expand=True)
        crit_df['table']=s[0]
        crit_df['column']=s[1]
        crit_df=crit_df[['table','column','criterion_value','criterion_operation']]

        crit_df.columns=['table','statistic','threshold','operation']

        if latex:
            crit_df.loc[crit_df['operation'].str.contains('<'),'operation']='maximum'
            crit_df.loc[crit_df['operation'].str.contains('>'),'operation']='minimum'
            crit_df.loc[crit_df['operation']=='=','operation']='equal to'
            info_out = open(out_file, 'w+', errors="backslashreplace")
            info_out.write('\documentclass{article}\n')
            info_out.write('\\usepackage{booktabs}\n')
            #info_out.write('\\usepackage{longtable}\n')
            info_out.write('\\begin{document}')
            info_out.write(crit_df.to_latex(index=False,longtable=False,
                                            escape=True,multicolumn=False))
            info_out.write('\end{document}\n')
            info_out.close()
        else:
            crit_df.to_excel(out_file,index=False)

    else: 
        print ("No criteria   for ouput.")
       

In [79]:
# latex way:
criteria_extract(output_dir_path='/users/ltauxe/Desktop',latex=True,output_file='criteria.tex',
              input_dir_path='data_files/3_0/Megiddo')

In [80]:
# excel way:
criteria_extract(output_dir_path='/users/ltauxe/Desktop',
              input_dir_path='data_files/3_0/Megiddo')

In [143]:
reload(map_magic)
def specimens_extract(spec_file='specimens.txt',output_file='specimens.xls',landscape=False,
                  longtable=False,output_dir_path='./',input_dir_path='./',latex=False):
    """
    Extracts specimen results  from a MagIC 3.0 format specimens.txt file.
    Default output format is an Excel file. 
    typeset with latex on your own computer.  
    
    Parameters
    ___________
    spec_file : str
         input file name
    input_dir_path : str
          path for intput file if different from output_dir_path (default is same)
    latex : boolean
           if True, output file should be latex formatted table with a .tex ending
    longtable : boolean
           if True output latex longtable
    Return :
        [True,False],  data table error type : True if successful
        
    Effects : 
        writes cvs or latex formatted tables for use in publications
    """
    if not input_dir_path:
        input_dir_path = output_dir_path
    try:
        fname = pmag.resolve_file_name(spec_file, input_dir_path)
    except IOError:
        print("bad specimen file name")
        return False, "bad specimen file name"
    spec_df=pd.read_csv(fname,sep='\t',header=1)
    spec_df.dropna('columns',how='all',inplace=True)
    spec_df.dropna(subset=['int_abs'],inplace=True)
    if len(spec_df)>0:
        table_df=map_magic.convert_specimen_dm3_table(spec_df)
        out_file=pmag.resolve_file_name(output_file,output_dir_path)
        if latex:
            info_out = open(out_file, 'w+', errors="backslashreplace")
            info_out.write('\documentclass{article}\n')
            info_out.write('\\usepackage{booktabs}\n')
            if landscape:info_out.write('\\usepackage{lscape}')
            if longtable:info_out.write('\\usepackage{longtable}\n')
            info_out.write('\\begin{document}\n')
            if landscape:info_out.write('\\begin{landscape}\n')
            info_out.write(table_df.to_latex(index=False,longtable=longtable,
                                            escape=True,multicolumn=False))
            if landscape:info_out.write('\end{landscape}\n')
            info_out.write('\end{document}\n')
            info_out.close()
        else:
            table_df.to_excel(out_file,index=False)

    else: 
        print ("No specimen data   for ouput.")

    


In [144]:
# latex way: 
specimens_extract(output_dir_path='/users/ltauxe/Desktop',output_file='specimens.tex',landscape=True,
              input_dir_path='data_files/3_0/Megiddo',latex=True,longtable=True)

-I- Using online data model


In [140]:
# excel way:
specimens_extract(output_dir_path='/users/ltauxe/Desktop',
              input_dir_path='data_files/3_0/Megiddo')

-I- Using online data model


In [8]:
def locations_extract(loc_file='locations.txt',output_file='locations.xls',
                  output_dir_path='./',input_dir_path='./',latex=False):
    """
    Extracts location averages  from a MagIC 3.0 format locations.txt file.
    Default output format is an Excel file. 
    typeset with latex on your own computer.  
    
    Parameters
    ___________
    loc_file : str
         input file name
    input_dir_path : str
          path for intput file if different from output_dir_path (default is same)
    latex : boolean
           if True, output file should be latex formatted table with a .tex ending
           
    Return :
        [True,False],  data table error type : True if successful
        
    Effects : 
        writes cvs or latex formatted tables for use in publications
    """
    if not input_dir_path:
        input_dir_path = output_dir_path
    try:
        fname = pmag.resolve_file_name(loc_file, input_dir_path)
    except IOError:
        print("bad location file name")
        return False, "bad location file name"
    loc_df=pd.read_csv(fname,sep='\t',header=1)
    if len(loc_df)>0:
        out_file=pmag.resolve_file_name(output_file,output_dir_path)
        poles=loc_df.dropna(subset=['pole_lat','pole_lon'])
        print (poles)
       

In [10]:
# excel way:
locations_extract(output_dir_path='/users/ltauxe/Desktop',
              input_dir_path='data_files/3_0/McMurdo')

  location                                              sites     age  \
1  McMurdo  mc03 : mc04 : mc06 : mc07 : mc08 : mc09 : mc10...  2.5620   
2  McMurdo  mc102 : mc103 : mc105 : mc109 : mc110 : mc112 ...  2.5540   
3  McMurdo  mc02 : mc03 : mc04 : mc06 : mc07 : mc08 : mc09...  2.5589   

   age_sigma age_unit  dir_alpha95  dir_dec  dir_inc  dir_k  dir_n_sites  \
1     2.8348       Ma          3.3     16.3    -80.8   26.0         75.0   
2     1.8641       Ma          4.1    186.8     77.7   27.2         46.0   
3     2.5017       Ma          2.5     12.1    -79.7   26.4        121.0   

      ...      pole_alpha95  pole_antipodal_angle pole_lat pole_lon  \
1     ...               5.5                   NaN     84.6    214.5   
2     ...               6.9                   NaN    -79.9      2.7   
3     ...               4.3                 173.8     83.1    197.5   

   pole_n_sites  pole_reversed_perc   citations   method_codes  \
1          75.0                 0.0  This study    

In [ ]:


         if latex:
            info_out = open(out_file, 'w+', errors="backslashreplace")
            info_out.write('\documentclass{article}\n')
            info_out.write('\\usepackage{booktabs}\n')
            #info_out.write('\\usepackage{longtable}\n')
            info_out.write('\\begin{document}')
            info_out.write(loc_df.to_latex(index=False,longtable=False,
                                            escape=True,multicolumn=False))
            info_out.write('\end{document}\n')
            info_out.close()
        else:
            loc_df.to_excel(out_file,index=False)

    else: 
        print ("No locations   for ouput.")
